# Minnesota CTC Reform Analysis (2025)

This notebook analyzes the impact of modifying Minnesota's Child Tax Credit (CWFC).

## Baseline (Current Law)
- CTC amount: $1,750 per qualifying child
- Main phase-out rate: 12%
- Phase-out rate for CTC-ineligible with older children: 9%

## Reform
- CTC amount: $2,000 per qualifying child
- Main phase-out rate: 20%
- Phase-out rate for CTC-ineligible with older children: 9% (unchanged)

## Metrics
We calculate:
- Budgetary impact (net cost)
- Winners (percentage of population affected)
- Overall poverty impact
- Child poverty impact

In [12]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

MN_DATASET = "hf://policyengine/policyengine-us-data/states/MN.h5"

## Helper Functions

In [13]:
def calculate_poverty(sim, period=2025, child_only=False):
    age = np.array(sim.calculate("age", period=period))
    is_in_poverty = np.array(sim.calculate("person_in_poverty", period=period))
    person_weight = np.array(sim.calculate("person_weight", period=period))
    
    if child_only:
        mask = age < 18
    else:
        mask = np.ones_like(age, dtype=bool)
    
    weighted_in_poverty = (is_in_poverty[mask] * person_weight[mask]).sum()
    weighted_total = person_weight[mask].sum()
    poverty_rate = weighted_in_poverty / weighted_total if weighted_total > 0 else 0
    
    return {
        "poverty_rate": poverty_rate,
        "people_in_poverty": weighted_in_poverty,
        "total_people": weighted_total
    }

def calculate_winners(baseline_sim, reform_sim, period=2025):
    baseline_income = np.array(baseline_sim.calculate("household_net_income", period=period, map_to="household"))
    reform_income = np.array(reform_sim.calculate("household_net_income", period=period, map_to="household"))
    household_weight = np.array(baseline_sim.calculate("household_weight", period=period))
    income_change = reform_income - baseline_income
    
    household_id_person = np.array(baseline_sim.calculate("household_id", period=period, map_to="person"))
    household_id_household = np.array(baseline_sim.calculate("household_id", period=period, map_to="household"))
    person_weight = np.array(baseline_sim.calculate("person_weight", period=period))
    
    income_change_dict = dict(zip(household_id_household, income_change))
    person_income_change = np.array([income_change_dict.get(hh_id, 0) for hh_id in household_id_person])
    
    winners_mask = person_income_change > 1
    people_winning = person_weight[winners_mask].sum()
    total_people = person_weight.sum()
    
    losers_mask = person_income_change < -1
    people_losing = person_weight[losers_mask].sum()
    
    pct_winners = (people_winning / total_people * 100) if total_people > 0 else 0
    pct_losers = (people_losing / total_people * 100) if total_people > 0 else 0
    
    winning_hh_mask = income_change > 1
    avg_gain = np.average(income_change[winning_hh_mask], weights=household_weight[winning_hh_mask]) if winning_hh_mask.sum() > 0 else 0
    
    losing_hh_mask = income_change < -1
    avg_loss = np.average(income_change[losing_hh_mask], weights=household_weight[losing_hh_mask]) if losing_hh_mask.sum() > 0 else 0
    
    return {
        "people_winning": people_winning,
        "people_losing": people_losing,
        "total_people": total_people,
        "pct_winners": pct_winners,
        "pct_losers": pct_losers,
        "avg_gain": avg_gain,
        "avg_loss": avg_loss
    }

def format_currency(value):
    return f"${value/1e6:.2f}M"

def format_percent(value):
    return f"{value*100:.2f}%"

## Define Baseline and Reform

In [14]:
def create_mn_ctc_reform():
    reform = Reform.from_dict(
        {
            "gov.states.mn.tax.income.credits.cwfc.ctc.amount": {
                "2025-01-01.2100-12-31": 2000
            },
            "gov.states.mn.tax.income.credits.cwfc.phase_out.rate.main": {
                "2025-01-01.2100-12-31": 0.20
            },
        },
        country_id="us",
    )
    return reform

print("Reform function defined!")
print("\nReform details:")
print("  - CTC amount: $1,750 -> $2,000 (+$250 per child)")
print("  - Main phase-out rate: 12% -> 20% (+8 percentage points)")

Reform function defined!

Reform details:
  - CTC amount: $1,750 -> $2,000 (+$250 per child)
  - Main phase-out rate: 12% -> 20% (+8 percentage points)


## Load Simulations

In [15]:
print("Loading baseline (current law)...")
baseline = Microsimulation(dataset=MN_DATASET)
print("Baseline loaded")

print("\nLoading reform (CTC $2,000 + 20% phase-out rate)...")
reform = create_mn_ctc_reform()
reform_sim = Microsimulation(dataset=MN_DATASET, reform=reform)
print("Reform loaded")

print("\n" + "="*60)
print("All simulations ready!")
print("="*60)

Loading baseline (current law)...
Baseline loaded

Loading reform (CTC $2,000 + 20% phase-out rate)...
Reform loaded

All simulations ready!


## Calculate Impacts

In [16]:
baseline_overall_pov = calculate_poverty(baseline, child_only=False)
baseline_child_pov = calculate_poverty(baseline, child_only=True)

reform_overall_pov = calculate_poverty(reform_sim, child_only=False)
reform_child_pov = calculate_poverty(reform_sim, child_only=True)

baseline_hh_income = baseline.calculate("household_net_income", period=2025, map_to="household").sum()
reform_hh_income = reform_sim.calculate("household_net_income", period=2025, map_to="household").sum()
ctc_cost = reform_hh_income - baseline_hh_income

winners = calculate_winners(baseline, reform_sim)

print("All impacts calculated")

All impacts calculated


## Results Summary

In [17]:
print("\n" + "="*80)
print("MINNESOTA CTC REFORM IMPACTS (2025)")
print("Baseline: CTC $1,750 + 12% phase-out | Reform: CTC $2,000 + 20% phase-out")
print("="*80)

print(f"\n{'BUDGETARY IMPACT':=^80}")
print(f"MN CTC Reform net cost:        {format_currency(ctc_cost)}")

print(f"\n{'WINNERS AND LOSERS (POPULATION)':=^80}")
print(f"People gaining income:         {winners['people_winning']:,.0f} ({winners['pct_winners']:.2f}% of population)")
print(f"Average gain per household:    ${winners['avg_gain']:,.2f}")
print(f"People losing income:          {winners['people_losing']:,.0f} ({winners['pct_losers']:.2f}% of population)")
print(f"Average loss per household:    ${winners['avg_loss']:,.2f}")

print(f"\n{'POVERTY IMPACT - OVERALL':=^80}")
print(f"Baseline poverty rate:         {format_percent(baseline_overall_pov['poverty_rate'])}")
print(f"Reform poverty rate:           {format_percent(reform_overall_pov['poverty_rate'])}")
overall_pov_reduction = baseline_overall_pov['poverty_rate'] - reform_overall_pov['poverty_rate']
overall_pov_pct_reduction = (overall_pov_reduction / baseline_overall_pov['poverty_rate'] * 100) if baseline_overall_pov['poverty_rate'] > 0 else 0
print(f"Absolute reduction:            {format_percent(overall_pov_reduction)}")
print(f"Relative reduction:            {overall_pov_pct_reduction:.2f}%")
people_lifted = baseline_overall_pov['people_in_poverty'] - reform_overall_pov['people_in_poverty']
print(f"People lifted from poverty:    {people_lifted:,.0f}")

print(f"\n{'POVERTY IMPACT - CHILDREN':=^80}")
print(f"Baseline child poverty rate:   {format_percent(baseline_child_pov['poverty_rate'])}")
print(f"Reform child poverty rate:     {format_percent(reform_child_pov['poverty_rate'])}")
child_pov_reduction = baseline_child_pov['poverty_rate'] - reform_child_pov['poverty_rate']
child_pov_pct_reduction = (child_pov_reduction / baseline_child_pov['poverty_rate'] * 100) if baseline_child_pov['poverty_rate'] > 0 else 0
print(f"Absolute reduction:            {format_percent(child_pov_reduction)}")
print(f"Relative reduction:            {child_pov_pct_reduction:.2f}%")
children_lifted = baseline_child_pov['people_in_poverty'] - reform_child_pov['people_in_poverty']
print(f"Children lifted from poverty:  {children_lifted:,.0f}")
print("="*80)


MINNESOTA CTC REFORM IMPACTS (2025)
Baseline: CTC $1,750 + 12% phase-out | Reform: CTC $2,000 + 20% phase-out

================================BUDGETARY IMPACT================================
MN CTC Reform net cost:        $24.44M

========================WINNERS AND LOSERS (POPULATION)=========================
People gaining income:         714,961 (17.58% of population)
Average gain per household:    $441.15
People losing income:          718,701 (17.67% of population)
Average loss per household:    $-486.30

============================POVERTY IMPACT - OVERALL============================
Baseline poverty rate:         20.87%
Reform poverty rate:           20.73%
Absolute reduction:            0.14%
Relative reduction:            0.69%
People lifted from poverty:    5,814

===========================POVERTY IMPACT - CHILDREN============================
Baseline child poverty rate:   23.92%
Reform child poverty rate:     23.53%
Absolute reduction:            0.39%
Relative reduction:

In [18]:
baseline_hh_income_arr = np.array(baseline.calculate("household_net_income", period=2025, map_to="household"))
reform_hh_income_arr = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="household"))
household_weight = np.array(baseline.calculate("household_weight", period=2025))

hh_income_change = reform_hh_income_arr - baseline_hh_income_arr
hh_benefitting_mask = hh_income_change > 1
hh_losing_mask = hh_income_change < -1

households_benefitting = household_weight[hh_benefitting_mask].sum()
households_losing = household_weight[hh_losing_mask].sum()
total_households = household_weight.sum()
pct_households_benefitting = (households_benefitting / total_households) * 100
pct_households_losing = (households_losing / total_households) * 100

print("="*70)
print("HOUSEHOLDS IMPACTED BY MN CTC REFORM")
print("="*70)
print(f"Households benefitting:        {households_benefitting:,.0f} ({pct_households_benefitting:.2f}%)")
print(f"Households losing:             {households_losing:,.0f} ({pct_households_losing:.2f}%)")
print(f"Total households:              {total_households:,.0f}")
print("="*70)

HOUSEHOLDS IMPACTED BY MN CTC REFORM
Households benefitting:        174,192 (13.88%)
Households losing:             107,771 (8.59%)
Total households:              1,254,857


## Export Results

In [19]:
overall_pov_reduction = baseline_overall_pov['poverty_rate'] - reform_overall_pov['poverty_rate']
overall_pov_pct_reduction = (overall_pov_reduction / baseline_overall_pov['poverty_rate'] * 100) if baseline_overall_pov['poverty_rate'] > 0 else 0
child_pov_reduction = baseline_child_pov['poverty_rate'] - reform_child_pov['poverty_rate']
child_pov_pct_reduction = (child_pov_reduction / baseline_child_pov['poverty_rate'] * 100) if baseline_child_pov['poverty_rate'] > 0 else 0

results = [
    {
        "Scenario": "MN CTC Reform",
        "Description": "CTC $2,000 + 20% phase-out rate",
        "Net Cost": format_currency(ctc_cost),
        "% Population Winning": f"{winners['pct_winners']:.2f}%",
        "% Population Losing": f"{winners['pct_losers']:.2f}%",
        "Avg Gain (Winners)": f"${winners['avg_gain']:,.2f}",
        "Avg Loss (Losers)": f"${winners['avg_loss']:,.2f}",
        "Overall Poverty Change (%)": f"{overall_pov_pct_reduction:.2f}%",
        "Child Poverty Change (%)": f"{child_pov_pct_reduction:.2f}%",
        "People Lifted from Poverty": f"{people_lifted:,.0f}",
        "Children Lifted from Poverty": f"{children_lifted:,.0f}"
    }
]

df_results = pd.DataFrame(results)

print("\n" + "="*120)
print("MN CTC REFORM SUMMARY")
print("="*120)
print(df_results.T.to_string(header=False))
print("="*120)

df_results.to_csv("mn_ctc_reform_results.csv", index=False)
print("\nExported to: mn_ctc_reform_results.csv")


MN CTC REFORM SUMMARY
Scenario                                        MN CTC Reform
Description                   CTC $2,000 + 20% phase-out rate
Net Cost                                              $24.44M
% Population Winning                                   17.58%
% Population Losing                                    17.67%
Avg Gain (Winners)                                    $441.15
Avg Loss (Losers)                                    $-486.30
Overall Poverty Change (%)                              0.69%
Child Poverty Change (%)                                1.62%
People Lifted from Poverty                              5,814
Children Lifted from Poverty                            4,544

Exported to: mn_ctc_reform_results.csv


## Average Income by Decile - Baseline vs Reform

In [20]:
baseline_income = np.array(baseline.calculate("household_net_income", period=2025, map_to="household"))
reform_income = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="household"))
weights = np.array(baseline.calculate("household_weight", period=2025))
agi = np.array(baseline.calculate("adjusted_gross_income", period=2025, map_to="household"))

df_decile = pd.DataFrame({
    'agi': agi,
    'baseline_income': baseline_income,
    'reform_income': reform_income,
    'weight': weights
})

df_decile = df_decile.sort_values('agi').reset_index(drop=True)
df_decile['cumweight'] = df_decile['weight'].cumsum()
total_weight = df_decile['weight'].sum()

df_decile['decile'] = pd.cut(
    df_decile['cumweight'] / total_weight,
    bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    labels=['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']
)

decile_summary = df_decile.groupby('decile', observed=True).apply(
    lambda x: pd.Series({
        'Baseline Avg Income': np.average(x['baseline_income'], weights=x['weight']),
        'Reform Avg Income': np.average(x['reform_income'], weights=x['weight']),
        'Avg Change': np.average(x['reform_income'] - x['baseline_income'], weights=x['weight']),
        'Households': x['weight'].sum()
    })
).reset_index()

print("\n" + "="*90)
print("AVERAGE HOUSEHOLD NET INCOME BY DECILE - BASELINE VS REFORM")
print("="*90)
print(f"{'Decile':>8} {'Baseline':>18} {'Reform':>18} {'Change':>14} {'Households':>14}")
print("-"*90)
for _, row in decile_summary.iterrows():
    print(f"{row['decile']:>8} ${row['Baseline Avg Income']:>16,.0f} ${row['Reform Avg Income']:>16,.0f} ${row['Avg Change']:>12,.2f} {row['Households']:>14,.0f}")
print("="*90)


AVERAGE HOUSEHOLD NET INCOME BY DECILE - BASELINE VS REFORM
  Decile           Baseline             Reform         Change     Households
------------------------------------------------------------------------------------------
     1st $          27,096 $          27,261 $      164.19        125,469
     2nd $          46,710 $          46,788 $       77.99        125,399
     3rd $          54,743 $          54,943 $      199.52        125,521
     4th $          75,031 $          74,936 $      -94.60        124,832
     5th $          87,036 $          86,945 $      -91.11        125,477
     6th $         128,949 $         129,003 $       54.59        126,104
     7th $         165,711 $         165,700 $      -11.68        125,395
     8th $         254,059 $         254,059 $       -0.48        123,740
     9th $         441,432 $         441,415 $      -17.61        127,447
    10th $         785,661 $         785,574 $      -86.57        125,472


## Winners, Losers, and Non-Affected by Decile

In [21]:
# Get person-level data with household mapping
person_weight = np.array(baseline.calculate("person_weight", period=2025))
household_id_person = np.array(baseline.calculate("household_id", period=2025, map_to="person"))
household_id_household = np.array(baseline.calculate("household_id", period=2025, map_to="household"))

# Get household-level income change
baseline_hh_inc = np.array(baseline.calculate("household_net_income", period=2025, map_to="household"))
reform_hh_inc = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="household"))
hh_inc_change = reform_hh_inc - baseline_hh_inc

# Get household AGI for decile assignment
hh_agi = np.array(baseline.calculate("adjusted_gross_income", period=2025, map_to="household"))
hh_weight = np.array(baseline.calculate("household_weight", period=2025))

# Create household-level DataFrame with decile assignment
df_hh = pd.DataFrame({
    'household_id': household_id_household,
    'agi': hh_agi,
    'income_change': hh_inc_change,
    'weight': hh_weight
})

# Sort by AGI and assign deciles
df_hh = df_hh.sort_values('agi').reset_index(drop=True)
df_hh['cumweight'] = df_hh['weight'].cumsum()
total_hh_weight = df_hh['weight'].sum()
df_hh['decile'] = pd.cut(
    df_hh['cumweight'] / total_hh_weight,
    bins=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    labels=['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th']
)

# Create mapping from household_id to decile and income_change
hh_to_decile = dict(zip(df_hh['household_id'], df_hh['decile']))
hh_to_change = dict(zip(df_hh['household_id'], df_hh['income_change']))

# Create person-level DataFrame
df_person = pd.DataFrame({
    'household_id': household_id_person,
    'person_weight': person_weight
})

# Map decile and income change to persons
df_person['decile'] = df_person['household_id'].map(hh_to_decile)
df_person['income_change'] = df_person['household_id'].map(hh_to_change)

# Classify as winner, loser, or non-affected
df_person['is_winner'] = df_person['income_change'] > 1
df_person['is_loser'] = df_person['income_change'] < -1
df_person['is_nonaffected'] = (~df_person['is_winner']) & (~df_person['is_loser'])

# Calculate percentages by decile
decile_impact = df_person.groupby('decile', observed=True).apply(
    lambda x: pd.Series({
        'Total Residents': x['person_weight'].sum(),
        'Winners': x.loc[x['is_winner'], 'person_weight'].sum(),
        'Losers': x.loc[x['is_loser'], 'person_weight'].sum(),
        'Non-Affected': x.loc[x['is_nonaffected'], 'person_weight'].sum()
    })
).reset_index()

# Calculate percentages
decile_impact['% Winners'] = (decile_impact['Winners'] / decile_impact['Total Residents'] * 100)
decile_impact['% Losers'] = (decile_impact['Losers'] / decile_impact['Total Residents'] * 100)
decile_impact['% Non-Affected'] = (decile_impact['Non-Affected'] / decile_impact['Total Residents'] * 100)

print("\n" + "="*90)
print("WINNERS, LOSERS, AND NON-AFFECTED BY INCOME DECILE (% OF RESIDENTS)")
print("="*90)
print(f"{'Decile':>8} {'% Winners':>12} {'% Losers':>12} {'% Non-Affected':>16} {'Total Residents':>18}")
print("-"*90)
for _, row in decile_impact.iterrows():
    print(f"{row['decile']:>8} {row['% Winners']:>11.1f}% {row['% Losers']:>11.1f}% {row['% Non-Affected']:>15.1f}% {row['Total Residents']:>17,.0f}")
print("="*90)


WINNERS, LOSERS, AND NON-AFFECTED BY INCOME DECILE (% OF RESIDENTS)
  Decile    % Winners     % Losers   % Non-Affected    Total Residents
------------------------------------------------------------------------------------------
    10th         1.6%        50.5%            47.9%           540,016
     1st        55.1%         0.3%            44.6%           301,218
     2nd        33.4%         0.0%            66.6%           242,554
     3rd        58.7%         0.0%            41.3%           334,197
     4th        14.0%        35.5%            50.5%           363,603
     5th        24.7%        20.0%            55.3%           396,382
     6th        21.5%         4.9%            73.6%           380,262
     7th         6.9%         9.6%            83.4%           438,273
     8th         0.4%         0.3%            99.2%           445,347
     9th         0.0%        28.0%            72.0%           624,459
